In [82]:
from __future__ import annotations
from typing import Tuple
import pandas as pd
import pickle
import numpy as np
from scipy import spatial
import sklearn as skl
import os
import json
import fasttext
import fasttext.util
import string
import math
from nltk.corpus import stopwords
import nltk
stops = set(stopwords.words('english'))
stops.add('notes')
stops.add('note')
stops.add('hint')
stops.add('hints')
wmd = fasttext.load_model('../glove/wiki.en/wiki.en.bin')

In [126]:
country_dict = {}
eng = open('../data/countriesEN', 'r').readlines()
fin = open('../data/countriesFI', 'r').readlines()
for e,f in zip(eng, fin):
    country_dict[f.strip()] = e.strip().lower()

print(country_dict)


{'Argentiina': 'argentina', 'Australia': 'australia', 'Bolivia': 'bolivia', 'Bulgaria': 'bulgaria', 'Chile': 'chile', 'Englanti': 'english', 'Espanja': 'spain', 'Etelä-Afrikka': 'south africa', 'Euroopan unioni': 'european union', 'Georgia': 'georgia', 'Intia': 'india', 'Israel': 'israel', 'Italia': 'italy', 'Itävalta': 'austria', 'Kanada': 'canada', 'Kiina': 'china', 'Kreikka': 'greece', 'Kroatia': 'croatia', 'Libanon': 'lebanon', 'Luxemburg': 'luxembourg', 'Meksiko': 'mexico', 'Moldova': 'moldova', 'Montenegro': 'montenegro', 'Muu alkuperämaa': 'other country of origin', 'Peru': 'peru', 'Portugali': 'portugal', 'Ranska': 'france', 'Romania': 'romania', 'Saksa': 'germany', 'Serbia': 'serbia', 'Slovakia': 'slovakia', 'Slovenia': 'slovenia', 'Sveitsi': 'switzerland', 'Tsekki': 'czech republic', 'Turkki': 'turkey', 'Unkari': 'hungary', 'Uruguay': 'uruguay', 'Uusi-Seelanti': 'new zealand', 'Venäjä': 'russia', 'Yhdysvallat': 'united states', 'Japani': 'japan', 'Kypros': 'cyprus', 'Pohjois-

In [131]:
def partition_words(desc: str, ) -> tuple[str]:
    #print(desc)
    terms = process(desc).split(',')
    #print(terms)
    try: 
        color = terms[0]
        #print(color)
        mouthfeel = terms[1]
        #print(mouthfeel)
        taste = terms[2:]
        #print(taste)
        return color, mouthfeel, taste
    except IndexError:
        return "", "", ""
def process(desc: str) -> str:
    tempTerms = []
    desc = desc.lower().strip()
    mapping = desc.maketrans({'-': '_'})
    desc = desc.translate(mapping)
    for terms in desc.split(','):
        #print(terms)
        term = ' '.join([ t for t in terms.split() if (t not in stops)])
        #print(term)
        tempTerms.append(term)
    #print(tempTerms)
    return ','.join(tempTerms)

def computeTasteVect(desc: tuple) -> np.ndarray:
    try: 
        col = desc[2]
    except IndexError:
        return np.nan
    tasteVec = np.zeros(300)
    for ta in taste:
        #for each word in the taste description, fetch the word vector and sum them all together
        tasteVec = np.add(tasteVec, wmd.get_word_vector(ta))
    #store the summed vector representation in beer_taste_vectors dic
    return tasteVec

def computeColorVect(desc: tuple) -> np.ndarray:
    try: 
        col = desc[0]
    except IndexError:
        return np.nan
    colVec = np.zeros(300)
    return np.add(colVec, wmd.get_word_vector(col))

def computeFeelVect(desc: tuple) -> np.ndarray:
    try: 
        feel = desc[1]
    except IndexError:
        return np.nan
    feelVec = np.zeros(300)
    return np.add(feelVec, wmd.get_word_vector(feel))

def computeCountryVect(maa: str) -> np.ndarray:
    return wmd.get_word_vector(maa)

In [79]:
#load the beer data from the json
#for each beer, 
bdf = pd.read_json(open('../data/beer_foods_tastes.json','r'), orient='index')
bdf.head()

,foods,taste_desc
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea..."
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich..."
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric..."
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri..."
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho..."


In [80]:
bdf['taste_desc_partition'] = bdf['taste_desc'].apply(partition_words)
bdf.head()

,foods,taste_desc,taste_desc_partition
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp..."
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head..."
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea..."
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he..."
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ..."


In [98]:
bdf['taste_vect'] = bdf['taste_desc_partition'].apply(computeTasteVect)
bdf['col_vect'] = bdf['taste_desc_partition'].apply(computeColorVect)
bdf['feel_vect'] = bdf['taste_desc_partition'].apply(computeFeelVect)
bdf.index = bdf.index.rename('Numero')
bdf.head()

,foods,taste_desc,taste_desc_partition,taste_vect,col_vect,feel_vect
Numero,,,,,,
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4940991997718811, -0.06226866692304611, 0....","[-0.2133956402540207, 0.22910979390144348, -0...."
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.11716967821121216, -0.06956495344638824, 0...","[-0.31225326657295227, -0.10067514330148697, 0..."
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.39861732721328735, -0.07806692272424698, -...","[-0.31225326657295227, -0.10067514330148697, 0..."
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0..."
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.31100010871887207, -0.18557031452655792, 0...","[-0.07209433615207672, 0.14588770270347595, 0...."


In [85]:
asdf = pickle.load(open('../data/country_vect_df.bin', 'rb'))

In [86]:
asdf.head()

,country_FI,country_EN,vect
0,Argentiina,Argentina,"[-0.1663632, 0.07589441, 0.32345265, -0.123166..."
1,Australia,Australia,"[-0.38573915, 0.08130928, -0.13291593, 0.13398..."
2,Bolivia,Bolivia,"[-0.25193572, -0.09912944, 0.1734743, -0.03175..."
3,Bulgaria,Bulgaria,"[-0.3960979, 0.10610346, -0.17874336, 0.508475..."
4,Chile,Chile,"[-0.027834663, -0.043989502, 0.10476164, 0.285..."


In [94]:
alkodf = pd.read_csv(open('../data/alkon-hinnasto-tekstitiedostona.csv','r'), sep = ',', index_col='Numero')
alkodf.head()

,Nimi,Valmistaja,Pullokoko,Hinta,Litrahinta,Uutuus,Hinnastojärjestyskoodi,Tyyppi,Alatyyppi,Erityisryhmä,...,Suljentatyyppi,Alkoholi-%,Hapot g/l,Sokeri g/l,Kantavierrep-%,Väri EBC,Katkerot EBU,Energia kcal/100 ml,Valikoima,EAN
Numero,,,,,,,,,,,,,,,,,,,,,
945096,Adrianna Vineyard Fortuna Terrae Malbec 2015,Catena Zapata,"0,75 l",84.96,113.28,NaN,110,punaviinit,Mehevä & Hilloinen,NaN,...,luonnonkorkki,14.0,5.6,2.0,NaN,NaN,NaN,80.0,tilausvalikoima,7794450005274
935635,Adrianna Vineyard River Stones Malbec 2017,Catena Zapata,"0,75 l",124.91,166.55,NaN,110,punaviinit,Mehevä & Hilloinen,NaN,...,luonnonkorkki,13.5,5.0,2.0,NaN,NaN,NaN,80.0,tilausvalikoima,7794450005304
440567,Alamos Malbec Organic 2020,Alamos,"0,75 l",10.99,14.52,NaN,110,punaviinit,Mehevä & Hilloinen,luomu,...,metallinen kierrekapseli,13.5,5.1,NaN,NaN,NaN,NaN,80.0,vakiovalikoima,7794450004871
424107,Alamos Tempranillo 2016,Alamos,"0,75 l",10.99,14.52,NaN,110,punaviinit,Mehevä & Hilloinen,NaN,...,metallinen kierrekapseli,12.5,4.6,2.0,NaN,NaN,NaN,70.0,vakiovalikoima,7794450092410
939055,Altos Las Hormigas Clásico Malbec 2018,Altos Las Hormigas,"0,75 l",16.99,22.65,NaN,110,punaviinit,Mehevä & Hilloinen,ympäristövastuullinen pakkaus,...,luonnonkorkki,13.5,5.1,3.0,NaN,NaN,NaN,80.0,tilausvalikoima,7798051950032


In [120]:
bdf.rename(columns={'Valmistusmaa':'country_FI'}, inplace=True)
bdf.head()

,foods,taste_desc,taste_desc_partition,taste_vect,col_vect,feel_vect,country_FI
Numero,,,,,,,
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4940991997718811, -0.06226866692304611, 0....","[-0.2133956402540207, 0.22910979390144348, -0....",Australia
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.11716967821121216, -0.06956495344638824, 0...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.39861732721328735, -0.07806692272424698, -...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.31100010871887207, -0.18557031452655792, 0...","[-0.07209433615207672, 0.14588770270347595, 0....",Belgia


In [129]:
bdf['country_EN'] = bdf['country_FI'].apply(lambda x: country_dict[x])
bdf.head()

,foods,taste_desc,taste_desc_partition,taste_vect,col_vect,feel_vect,country_FI,country_EN
Numero,,,,,,,,
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4940991997718811, -0.06226866692304611, 0....","[-0.2133956402540207, 0.22910979390144348, -0....",Australia,australia
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.11716967821121216, -0.06956495344638824, 0...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.39861732721328735, -0.07806692272424698, -...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.31100010871887207, -0.18557031452655792, 0...","[-0.07209433615207672, 0.14588770270347595, 0....",Belgia,belgium


In [132]:
bdf['country_vect'] = bdf['country_EN'].apply(computeCountryVect)
bdf.head()

,foods,taste_desc,taste_desc_partition,taste_vect,col_vect,feel_vect,country_FI,country_EN,country_vect
Numero,,,,,,,,,
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4940991997718811, -0.06226866692304611, 0....","[-0.2133956402540207, 0.22910979390144348, -0....",Australia,australia,"[-0.2850604, -0.03134475, -0.22727826, 0.54968..."
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.11716967821121216, -0.06956495344638824, 0...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698..."
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.39861732721328735, -0.07806692272424698, -...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698..."
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698..."
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.31100010871887207, -0.18557031452655792, 0...","[-0.07209433615207672, 0.14588770270347595, 0....",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698..."


In [135]:
bdf = bdf.join(alkodf[['Kantavierrep-%','Väri EBC','Katkerot EBU', 'Litrahinta']], 'Numero')

In [136]:
bdf.head()

,foods,taste_desc,taste_desc_partition,taste_vect,col_vect,feel_vect,country_FI,country_EN,country_vect,Kantavierrep-%,Väri EBC,Katkerot EBU,Litrahinta
Numero,,,,,,,,,,,,,
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4940991997718811, -0.06226866692304611, 0....","[-0.2133956402540207, 0.22910979390144348, -0....",Australia,australia,"[-0.2850604, -0.03134475, -0.22727826, 0.54968...",26.6,NaN,NaN,23.92
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.11716967821121216, -0.06956495344638824, 0...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",17.8,15.9,23.0,15.18
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.39861732721328735, -0.07806692272424698, -...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",18.5,41.8,24.0,16.18
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",19.4,70.5,18.0,20.68
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.31100010871887207, -0.18557031452655792, 0...","[-0.07209433615207672, 0.14588770270347595, 0....",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",17.6,NaN,7.0,16.42


In [141]:
bdf

,foods,taste_desc,taste_desc_partition,taste_vect,col_vect,feel_vect,country_FI,country_EN,country_vect,Kantavierrep-%,Väri EBC,Katkerot EBU,Litrahinta
Numero,,,,,,,,,,,,,
718897,"[Tapas_ja_antipasti, Seurustelujuoma, Miedot_j...","Golden-yellow, acidic, ripe apricot notes, tea...","(golden_yellow, acidic, [ripe apricot, tea, sp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4940991997718811, -0.06226866692304611, 0....","[-0.2133956402540207, 0.22910979390144348, -0....",Australia,australia,"[-0.2850604, -0.03134475, -0.22727826, 0.54968...",26.6,NaN,NaN,23.92
715894,"[Grilliruoka, Kana_kalkkuna, Miedot_juustot, P...","Amber-yellow, full-bodied, cloudy, with a rich...","(amber_yellow, full_bodied, [cloudy, rich head...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.11716967821121216, -0.06956495344638824, 0...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",17.8,15.9,23.0,15.18
758594,"[Rasvainen_kala, Miedot_juustot, Nautiskelujuo...","Caramel-brown, full-bodied, cloudy, with a ric...","(caramel_brown, full_bodied, [cloudy, rich hea...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.39861732721328735, -0.07806692272424698, -...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",18.5,41.8,24.0,16.18
730097,"[Voimakkaat_juustot, Riistalinnut, Nautiskeluj...","Mahogany-brown, full-bodied, cloudy, with a ri...","(mahogany_brown, full_bodied, [cloudy, rich he...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",19.4,70.5,18.0,20.68
919855,"[Ayriaiset, Vaharasvainen_kala, Nautiskelujuoma]","Straw-yellow, medium-bodied, cloudy, mildly ho...","(straw_yellow, medium_bodied, [cloudy, mildly ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.31100010871887207, -0.18557031452655792, 0...","[-0.07209433615207672, 0.14588770270347595, 0....",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",17.6,NaN,7.0,16.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911916,"[Seurustelujuoma, Nautiskelujuoma]","Mahogany-brown, full-bodied, strongly hopped, ...","(mahogany_brown, full_bodied, [strongly hopped...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.4017575979232788, 0.04059884324669838, 0.1...","[-0.31225326657295227, -0.10067514330148697, 0...",Yhdysvallat,united states,"[0.045622084, -0.35862827, 0.1916659, 0.238043...",26.5,84.5,55.0,45.26
935551,"[Riista, Nautiskelujuoma, Suklaaherkut]","Black, extra full-bodied, medium hopped, espre...","(black, extra full_bodied, [medium hopped, esp...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.3380114436149597, -0.0010778139112517238, ...","[-0.15684814751148224, -0.03309787064790726, -...",Yhdysvallat,united states,"[0.045622084, -0.35862827, 0.1916659, 0.238043...",26.0,NaN,49.0,40.86
906362,"[Marjat_ja_hedelmat, Miedot_juustot, Nautiskel...","Yellowy brown, light-bodied, mildly hopped, ci...","(yellowy brown, light_bodied, [mildly hopped, ...","[0.10126800090074539, 0.29972461797297, -0.383...","[-0.24740217626094818, -0.14776425063610077, -...","[0.033542584627866745, -0.24642294645309448, 0...",Belgia,belgium,"[-0.19977358, -0.12839371, -0.2806692, 0.11698...",13.1,NaN,22.0,15.27
